In [1]:
import pandas as pd
import os
import numpy as np
import random
import cv2
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import dlib
from pathlib import Path
import sys, os

ModuleNotFoundError: No module named 'dlib'

In [6]:
class PreProc:

    def __init__(self):
        self.data = None
        
    def loadDataset(self, dirImg, formatFile, mapLabel):
        lst = glob.glob(dirImg + "/*." + formatFile)
        labels, dataPath = [], []
        l1, l1Norm = [], []
        net = init_model_dnn()
        strt_ind = lst[0]
        for file in lst:
            im = load_img(file, False, None)
            detections = find_faces_dnn(im, net=net)
            confidence_max_pos = np.argmax(detections[0, 0, :, 2])
            if detections[0, 0, confidence_max_pos, 2] > self.conf_threshold:
                (h, w) = im.shape[:2]
                box = detections[0, 0, 0, 3:7] * np.array([w, h, w, h]).astype("int")
                l1.append(box)
                l1.append(detections[0, 0, 0, 3:7])
                dataPath.append(file)
                labels.append(mapLabel[file[strt_ind : strt_ind + 2]])
        l1 = np.array(l1)
        l1Norm = np.array(l1Norm)
        self._jaffe_data = {'file': data_path, 'label': labels, 
                            'x0' : l1[:, 0], 'y0' : l1[:, 1], 
                            'x1' : l1[:, 2], 'y1' : l1[:, 3]}
        
    def clear(self):
        del self.data
        self.data = None

In [7]:
def haarCascadeLoad():
    return cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [9]:
def findFaces(img, cascades, param1=1.5, param2=5):
    return cascades.detectMultiScale(img, param1, param2)

In [12]:
def cutOutFaces(img, cascades, newSize):
    faces = findFaces(img, cascades)
    facesFound = []
    for (x, y, w, h) in faces:
        face = cv2.resize(img[y:y + h, x:x + w], newSize, interpolation=cv2.INTER_AREA)
        facesFound.append(face)
    return facesFound

In [14]:
def haarFacesDB(xData, newSize=None):
    cascades, newData = haarCascadeLoad(), []
    for img in xData:
        newData.append(np.array(cutOutFaces(np.array(img), cascades=cascades, newSize=newSize)))
    return newData

In [15]:
def cutOutFacesDNN(im, net, newSize, conf_threshold=0.98):
    faces_found = []
    h, w = im.shape[:2]
    net.setInput(cv2.dnn.blobFromImage(cv2.resize(im, (300, 300)), 1.0, (300, 300), [104, 117, 123], False, False))
    for i in range(0, net.forward().shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > conf_threshold:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            startX, startY, endX, endY = box.astype("int")
            face = cv2.cvtColor(im[startY:endY, startX:endX], cv2.COLOR_BGR2GRAY)
            faces_found.append(cv2.resize(face, newSize, interpolation=cv2.INTER_AREA))

    return np.array(faces_found)

In [16]:
def findFacesDNN(im, net):
    return net.setInput(cv2.dnn.blobFromImage(cv2.resize(im, (300, 300), interpolation=cv2.INTER_AREA), 1.0, (300, 300),
                                 [104, 117, 123], False, False)).forward()

In [18]:
def facesFromDNNDB(xData, net, newSize=None, to_greyscale=False):
    newData = []
    for img in xData:
        newData.append(np.array(cut_out_faces_dnn(np.array(img), net=net, newSize=newSize, to_greyscale=to_greyscale)))
    return newData

In [ ]:
from keras.optimizers import SGD, Adam
from keras.models import Sequential, model_from_json, Model 
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPool2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPool2D
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.densenet import DenseNet121
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.mobilenet import MobileNet

In [20]:
class EmNet:
    def __init__(self, lossFunc, optim):
        self.model = Sequential()
        self.optim  = optim
        self.lossFunc = lossFunc
        
    def model(self, shape, n):
        self._model.add(Conv2D(32, (3, 3), input_shape = shape, activation = 'relu'))
        self._model.add(Conv2D(32, (3, 3), input_shape = shape, activation = 'relu'))
        self._model.add(Conv2D(32, (3, 3), input_shape = shape, activation = 'relu'))
        self._model.add(BatchNormalization(epsilon=0.0001))
        self._model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
        
        self._model.add(Conv2D(64, (3, 3), input_shape = shape, activation = 'relu'))
        self._model.add(Conv2D(64, (3, 3), input_shape = shape, activation = 'relu'))
        self._model.add(Conv2D(64, (3, 3), input_shape = shape, activation = 'relu'))
        self._model.add(BatchNormalization(epsilon=0.0001))
        self._model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
        
        self._model.add(Conv2D(64, (3, 3), input_shape = shape, activation = 'relu'))
        self._model.add(Conv2D(64, (3, 3), input_shape = shape, activation = 'relu'))
        self._model.add(Conv2D(64, (3, 3), input_shape = shape, activation = 'relu'))
        self._model.add(BatchNormalization(epsilon=0.0001))
        self._model.add(GlobalAveragePooling2D())
        
        self._model.add(Dense(512, activation = "relu"))
        self._model.add(Dropout(0.5)) 
        self._model.add(Dense(256, activation = "relu"))
        self._model.add(Dropout(0.5))
        self._model.add(Dense(n, activation = 'softmax'))
        
    def train(self, x_train, y_train, batch, epochs, lossFunc, optim, save_best, save_best_to):

        self.lossFunc = loss_func
        self.optim  = optim
        self.model.compile(loss = loss_func, optimizer = optim, metrics = ["accuracy"])

        return self._model.fit(x_train, y_train, batch_size = batch, epochs = epochs, verbose = 1, shuffle = True, 
                        validation_split = 0.15)
                    
    def saveWeights(self, fileName):
        self.model.save_weights(fileName)


    def saveModel(self, fileName):
        with open(fileName, 'w') as json_file:
            json_file.write(model.to_json())

    def predict(self, imgs):
        self.model.compile(loss = self.lossFunc, optimizer = self.optim)
        return self.model.predict(imgs)

In [22]:
def normalize_data(x_data, y_data, im_rows, im_cols, channels=1):
        x_data = np.array(x_data, dtype='float32')
        y_data = np.array(y_data, dtype='int32')
        x_data = x_data.reshape((x_data - 123.0) / 123.0.shape[0], im_rows, im_cols, channels)
        return x_data, np_utils.to_categorical(y_data, n_classes), np.unique(y_data).shape[0]

In [ ]:
def trainModel(dataset_csv, 
                        im_shape, 
                        model_folder = 'models\\', 
                        augment=False,
                        detect_face=True,
                        model_id='', 
                        save_model_id = '',
                        load_weights=True, 
                        plot_metrix=False, 
                        channels=1,
                        epocs=50,
                        batch_size=32,
                        arc=0):

        gray = True if channels==1 else False
        im_rows, im_cols = im_shape
        if detect_face:
                x_data, y_data = load_dataset_and_cut_out_faces(dataset_csv, label_map, new_size=(im_rows, im_cols), greyscale=gray)
        else:
                x_data, y_data = load_dataset_no_face(dataset_csv, new_size=(im_rows, im_cols), greyscale=gray,
                                label_map = {'anger':2, 'surprise':3, 'neutral':0, 'happiness':1, 'sadness':4})
        input_shape = (im_rows, im_cols, channels)
        x_data, y_data, n_classes = normalize_data(x_data, y_data, im_rows, im_cols, channels)

        x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.15, random_state=random.seed())

        Model = Emotion_Net()

        if load_weights:
                Model.load_model(model_folder + model_id + "model.json")
                Model.load_weights(model_folder + model_id + "model.h5")
        else:
                Model.init_model(input_shape, n_classes, arc=arc)
        if augment:
                history_call = Model.augment_and_train(x_train, y_train,
                        save_best_to = model_folder + save_model_id + "model.h5", \
                        batch_size=batch_size, n_epochs=epocs, optim=Adam())
        else:
                history_call = Model.train(x_train, y_train, 
                        save_best=False,
                        save_best_to = model_folder + save_model_id + "model.h5", \
                        batch_size=batch_size, n_epochs=epocs, optim=Adam())
        Model.evaluate_accur(x_test, y_test)
        if plot_metrix:
                plt.subplot(2,1,1)
                plt.title('Loss')
                plt.plot(history_call.history['loss'], label='training loss')
                plt.plot(history_call.history['val_loss'], label='testing loss')

                plt.subplot(2,1,2)
                plt.title('Accuracy')
                plt.plot(history_call.history['acc'], label='training accuracy')
                plt.plot(history_call.history['val_acc'], label='training accuracy')
                
                plt.legend()

        Model.save_model(model_folder + save_model_id + "model.json")
        Model.save_weights(model_folder + save_model_id + "model.h5")
        return n_classes

In [ ]:
rows, cols = (144, 144)
    channels = 3
    n_classes = 5
    n_classes = train_keras_model(os.path.join(ROOT_DIR, 'data\\dataset.csv'), 
                                im_shape=(rows, cols), 
                                channels=channels, 
                                model_folder = os.path.join(ROOT_DIR, 'saved_models\\'),
                                epocs=60, 
                                batch_size=32,
                                augment=False, 
                                detect_face=True,
                                load_weights=False, 
                                model_id='resnet_', 
                                save_model_id='resnet_',
                                plot_metrix=True, 
                                arc=2)
    plt.show()